# Importing libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

# Generating the data

In [5]:
np.random.seed(42)

m = 100
n = 3

X = np.random.rand(m,n)

y = 3 * X[: , 0] + 2 * X[: , 1] + 4 * X[: , 2] + np.random.randn(m)

outlier_indices = np.random.choice(range(m) , size =10 , replace = False)
X[outlier_indices] = X[outlier_indices] * 10
y[outlier_indices] = y[outlier_indices] + 50

# Z-Score to remove outliers

In [7]:
def compute_zscore(X):

 mean =np.mean(X,axis=0)
 std =np.std(X,axis=0)
 zscore=(X-mean)/std

 return np.abs(zscore)



X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_scaled = (X -X_mean)/ X_std
X_augmented = np.c_[np.ones((m, 1)), X_scaled]

zscores = compute_zscore(X_augmented[: , 1:])


threshold= 3

non_outlier_indices = np.all(zscores < threshold, axis=1)

X_filtered = X_augmented[non_outlier_indices]
y_filtered = y[non_outlier_indices]


print(f'number of data before cleaning {len(y)}')
print(f'number of data after cleaning {len(y_filtered)}')

Z-Score 100
number of data before cleaning 100
number of data after cleaning 91


# Cost with L1 and L2 regularization

In [8]:
def compute_cost(X,y,W ,lambda_l1=0 , lambda_l2=0 ) :
    m = len(y)
    predictions = X.dot(W)
    resid = predictions - y

    l2_reg = (lambda_l2 / (2*m))*np.sum(W[1:]**2)
    l1_reg = (lambda_l1 /m) *np.sum(np.abs(W[1:]))


    cost = (1/2*m) * np.sum(resid**2) + l2_reg + l1_reg
    return cost

## GDC

In [19]:

def gdc_tracked(X, y, w, learning_rate, n_epochs, lambda_l1=0, lambda_l2=0):
    if y.ndim == 1:
        y = y.reshape(-1, 1)

    m, n = X.shape
    cost_history = []
    w_history = []

    for epoch in range(n_epochs):
        predictions = X @ w
        resid = predictions - y

        gradients = (1/m) * X.T @ resid

        if lambda_l1 > 0:
            gradients[1:] += (lambda_l1/m) * np.sign(w[1:]).reshape(-1, 1)

        if lambda_l2 > 0:
            gradients[1:] += (lambda_l2/m) * w[1:]

        w -= learning_rate * gradients

        cost = compute_cost(X, y, w, lambda_l1, lambda_l2)
        cost_history.append(cost)
        w_history.append(w.copy())

        if epoch % 50 == 0:
            print(f"Epoch {epoch}: Cost={cost:.4f}")
    return w, cost_history, w_history


# Training model

In [32]:
initial_weights = np.zeros((X_filtered.shape[1], 1))
learning_rate = 0.1
n_epochs = 500
lambda_l1 = 0.1
lambda_l2 = 0.1



weights, cost_history, weight_history = gdc_tracked(X_filtered, y_filtered, initial_weights, learning_rate, n_epochs, lambda_l1, lambda_l2)


Epoch 0: Cost=207400.5490


# Convergence and Evolution of Weights